In [1]:
import pandas as pd

In [3]:
links1_df = pd.read_parquet('../../data/raw/property_links.parquet')
links2_df = pd.read_parquet('../../data/raw/property_links_2.parquet')

In [11]:
links1_df.drop_duplicates(inplace=True)

In [12]:
links1_df

,property_link
0,https://www.domain.com.au/7-langley-drive-epso...
1,https://www.domain.com.au/150-minards-lane-lon...
2,https://www.domain.com.au/11-charlbert-place-s...
3,https://www.domain.com.au/15-serene-drive-stra...
4,https://www.domain.com.au/25-sanctuary-bouleva...
...,...
95624,https://www.domain.com.au/28-waratah-avenue-wa...
95660,https://www.domain.com.au/45-stanley-street-to...
95662,https://www.domain.com.au/17a-stanley-street-t...
96092,https://www.domain.com.au/14-charlton-rise-dal...
